# PPO in Gym Lunar Lander

This notebook will have the implementation for PPO applied to Gym environment for later using it to the Transformer problem.

In [1]:
import os, sys
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import math
import random as rd
import matplotlib.pyplot as plt
from scipy import integrate
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras import activations
from collections import deque
from tqdm import tqdm
import shutil
import time

In [2]:
import gym

In [3]:
tf.config.list_physical_devices('GPU')

[]

In [4]:
ENV_NAME = 'LunarLanderContinuous-v2'
CONTINUOUS = True

## Reinforcement Learning
In the section below, a reinforcement learning agent shall be implemented and trained to perform a landing in the environment defined above. The reward function is already given.

The agent that will be implemented in this notebook is a PPO agent. 

###  Actor Model

In [5]:
def get_value_network(input_size):
    '''
    In this case, the input should be the env.observation_space.shape[0]
    '''
    x = keras.Input(input_size)
    
    v = Dense(10*input_size[0], activation = 'relu')(x)
    v = Dense(30, activation = 'relu')(v)
    v = Dense(17, activation = 'relu')(v)
    v = Dense(5, activation = 'relu')(v)
    v = Dense(1, activation = 'linear')(v)
    
    model = keras.Model(x,v)
    
    return model

In [6]:
def get_policy_network(agent, input_size, output_size):
    '''
    Input: env.observation_space.shape 
    Output_size: should be env.action_space.shape[0]
    '''
    input_x = keras.Input(input_size)
    
    x = Dense(10*input_size[0],activation='relu')(input_x)
    x = Dense(100,activation='relu')(x)
    x = Dense(50,activation='relu')(x)
    x = Dense(10*output_size[0],activation='relu')(x)

    mu = Dense(output_size[0], activation = 'tanh')(x)
    log_std = Dense(output_size[0], activation = 'linear')(x) # We get the logarithm of the std

    model = keras.Model(input_x,[mu, log_std])
    
    return model   

In [7]:
def get_actions(mu, log_std):
    log_std = tf.clip_by_value(log_std, -9, 2)
    std = tf.exp(log_std)
    gaussian = tfp.distributions.Normal(loc = mu, scale = std)
    actions = gaussian.sample(1)
    
    #####
    #print('_ _ _ _ _ _')
    #print('GET ACTIONS')
    #print('actions shape', actions.shape)
    #print('actions before squeeze', actions)
    #print('_ _ _ _ _ _')
    #####
    
    actions = tf.squeeze(actions,axis=0) # Corrected (before, axis = 1)
    assert mu.shape == actions.shape, 'mu and actions shape is not the same, be careful with squeeze'
    actions = tf.clip_by_value(actions,-1,1)
    return actions 

## Losses

In [8]:
def calc_logprob(mu, log_std, action):
    
    #action = tf.squeeze(action,axis=1)
    action = tf.cast(action, dtype=tf.float32)
    
    assert mu.shape == action.shape, 'mu and actions shape is not the same in calcprob: action ' + str(action.shape) + ' mu ' + str(mu.shape)
    assert log_std.shape == action.shape, 'mu and actions shape is not the same in calcprob'
    
    #p1 = - tf.math.divide(((mu - action) ** 2),(2*tf.exp(log_std)))
    #p2 = - tf.math.log(tf.math.sqrt(2 * np.pi * tf.exp(log_std)))
    
    #assert p1.shape == p2.shape, 'p1 and p2 have different shapes'
    #assert not np.isinf(np.sum(p1.numpy())), 'p1 is infinite'
    #assert not np.isinf(np.sum(p2.numpy())), 'p2 is infinite'
    #assert tf.reduce_all(tf.math.is_nan(p1[0])) == False, "p1 have NaNs"
    #assert tf.reduce_all(tf.math.is_nan(p2[0]))== False, "p2 have NaNs"

    std = tf.exp(log_std) # Corrected
    gaussian = tfp.distributions.Normal(loc = mu, scale = std) # Corrected
    p3 = gaussian.log_prob(action) # Corrected
    
    assert not np.isinf(np.sum(p3.numpy())), 'log_prob is infinite'
    
    #####
    #print('CALC_LOGPROB')
    #print('action', action[0])
    #print('mu', mu.shape)
    #print('log_std', log_std.shape)
    #print('p1', p1.shape)
    #print('_______________')
    #if tf.math.is_nan(tf.math.reduce_sum(p1)):
        #print('p1',p1)
        #print('mu',mu)
    #if tf.math.is_nan(tf.math.reduce_sum(p2)):
        #print('p2',p2)
        #print('log_std',log_std)
    #####

    return p3 # Corrected (before, return p1+p2)

In [9]:
def get_adv_ref(agent, trajectory):
    # gives the adv and ref from a single trajectory
    gamma = agent.gamma
    gae_lambda = agent.gae_lambda

    states, actions, rewards, terminals, next_states = map(list, zip(*trajectory))
    
    #####
    #print('_ _ _ _ _ _ _')
    #print('GET_ADV_REF')
    #print('things when getting them from the trajectory')
    #print('states shape',np.shape(states))
    #print('states',states)
    #print('actions shape',np.shape(actions))
    #print('actions',actions)
    #print('rewards shape',np.shape(rewards))
    #print('rewards',rewards)
    #####
    
    states = np.concatenate(states)
    next_states = np.concatenate(next_states)
    rewards = np.array(rewards)
    
    #####
    #print('things after concatenating them:')
    #print('states shape',states.shape)
    #print('states',states)
    #print('rewards shape',rewards.shape)
    #print('rewards',rewards)
    #####
    
    value_v = agent.critic(states)
    
    #####
    #print('value function after NN')
    #print('value_v shape', value_v.shape)
    #print('value_v',value_v)
    #####
    
    values = tf.squeeze(value_v).numpy()

    next_value_v = agent.critic(next_states)
    next_values = tf.squeeze(next_value_v).numpy()

    #####
    #print('value function after squeeze')
    #print('value_v shape squeezed', value_v.shape)
    #print('value_v squeezed',value_v)
    #print('states when getting adv and ref',states.shape)
    #print('values', values.shape)
    #print('rewards in adv and ref', rewards.shape)
    #print('next_values', values.shape)
    #####

    # generalized advantage estimator: smoothed version of the advantage
    last_gae = 0.0
    result_adv = []
    result_ref = []
    
    for val, next_val, reward, terminal in zip(reversed(values),reversed(next_values),reversed(rewards),reversed(terminals)):
        
        #####
        #print('inside the loop')
        #print('val',val)
        #print('next_val', next_val)
        #print('reward',reward)
        #print('terminal',terminal)
        #####
        
        if terminal:
            delta = reward - val
            last_gae = delta
            
            #####
            #print('inside the if')
            #print('delta',delta)
            #print('last_gae',last_gae)
            #####
            
        else:
            delta = reward + gamma*next_val - val
            last_gae = delta + gamma*gae_lambda*last_gae
            
            #####
            #print('inside the else')
            #print('delta',delta)
            #print('last_gae',last_gae)
            #####

        result_adv.append(last_gae)
        result_ref.append(last_gae + val)

        #####
        #print('element in adv',last_gae)
        #print('element in ref', last_gae + val)
        #print('_______________')
        #####    
    
    #####
    #print('adv_v NOT reversed shape',np.shape(result_adv))
    #print('adv_v NOT reversed',result_adv)
    #print('ref_v NOT reversed shape',np.shape(result_ref))
    #print('ref_v NOT reversed',result_ref)
    #####
    
    adv_v = list(reversed(result_adv))
    ref_v = list(reversed(result_ref))
    
    #####
    #print('adv_v reversed shape',np.shape(adv_v))
    #print('adv_v reversed',adv_v)
    #print('ref_v reversed shape',np.shape(ref_v))
    #print('ref_v reversed',ref_v)
    #print('_ _ _ _ _ _ _')
    #####
    
    return adv_v, ref_v

In [10]:
def loss_critic(ref_val,value_v):
    loss = tf.keras.losses.MSE(value_v,ref_val)
    
    return loss

# Implementation

In [11]:
def optimize_network(agent,states,actions,adv_batch,ref_batch,batch_old_logprob_v):
    with tf.GradientTape() as tape1, tf.GradientTape() as tape2:

        # critic training
        value_v = agent.critic(states)
        assert ref_batch.shape == value_v.shape, "ref_batch and value_v have different shapes"
        loss_value_v = loss_critic(ref_batch,value_v)

        # actor training
        mu, log_std = agent.actor(states)
        log_std = tf.clip_by_value(log_std, -9, 2)
        logprob_pi_v = calc_logprob(mu,log_std,actions)
        assert logprob_pi_v.shape == batch_old_logprob_v.shape, "logprob_pi_v.shape and batch_old_logprob_v have different shapes"
        ratio_v = tf.exp(logprob_pi_v - batch_old_logprob_v) # only positive values
        surr_obj_v = tf.math.multiply(adv_batch,ratio_v)

        #####
        #print('OPTIMIZE NETWORK')
        #print('Critic')
        #print('value_v', value_v.shape)
        #print('ref_batch', ref_batch.shape)
        #print('loss_value_v', loss_value_v.shape)
        #print('Actor')
        #print('mu', mu.shape, 'log_std', log_std.shape)
        #print('states', states.shape)
        #print('actions', actions.shape)
        #print('log_prob_pi', logprob_pi_v.shape)
        #print('batch_old_logprob_v', batch_old_logprob_v.shape)
        #print('ratio_v', ratio_v.shape)
        #print('surr_obj_v',surr_obj_v.shape)
        #print('adv_batch',adv_batch.shape)
        #print('ref_batch',ref_batch.shape)
        #print('_______________')
        #####
        
        
        clipped_surr_v = tf.math.multiply(adv_batch,tf.clip_by_value(ratio_v, 1.0 - agent.ppo_eps, 1 + agent.ppo_eps))
        loss_policy_v = - tf.math.reduce_mean(tf.math.minimum(surr_obj_v,clipped_surr_v))
        
        #####
        #print('surr_obj_v',surr_obj_v.shape)
        #if loss_policy_v > 1.2:
            #print('ratio_v',tf.math.reduce_mean(ratio_v))
            #print('surr_obj_v',tf.math.reduce_mean(surr_obj_v))
            #print('loss',loss_policy_v)
        if tf.reduce_all(tf.math.is_nan(loss_policy_v)):
            print('adv_batch',adv_batch)
            print('ratio_v',ratio_v)
            print('clipped_ratio_v',clipped_ratio_v)
            print('logprob_pi',logprob_pi_v)
            print('batch_old_logprob_v',batch_old_logprob_v)
            print('mu',mu)
            print('log_std',log_std)
            print('actions',actions)
        #####
        
    grads_crit = tape1.gradient(loss_value_v, agent.critic.trainable_variables)
    grads_act = tape2.gradient(loss_policy_v, agent.actor.trainable_variables)
    
    grads_act, _ = tf.clip_by_global_norm(grads_act, 0.5)
    assert tf.reduce_all(tf.math.is_nan(grads_crit[0])) == False, "grads_crit have NaNs"
    assert tf.reduce_all(tf.math.is_nan(grads_act[0])) == False, "grads_act have NaNs"
    
    agent.critic_optimizer.apply_gradients(zip(grads_crit, agent.critic.trainable_variables))
    agent.actor_optimizer.apply_gradients(zip(grads_act, agent.actor.trainable_variables))

In [12]:
 class Agent():
    def __init__(self):
        self.name = 'A2C_Agent'
        
    def agent_init(self):
        self.name = 'A2C_Agent'
        self.state_size = STATE_SIZE
        self.output_size = OUTPUT_SIZE
        self.test_iterations = TEST_ITERATIONS
        self.batch_size = BATCH_SIZE
        self.critic_lr = ADAM_LR_CRITIC
        self.actor_lr = ADAM_LR_ACTOR
        self.gamma = GAMMA
        self.gae_lambda = GAE_LAMBDA
        self.tolerance = TOLERANCE
        self.buffer_size = BUFFER_SIZE
        self.timeout = TIMEOUT
        self.ppo_eps = PPO_EPS
        self.test_episode = TEST_EPISODE
        self.epochs = TRAIN_EPOCHS
        
        # Generate NNs
        self.actor = get_policy_network(self,self.state_size,self.output_size)
        self.actor_optimizer = keras.optimizers.Adam(lr = self.actor_lr)
        self.critic = get_value_network(self.state_size)
        self.critic_optimizer = keras.optimizers.Adam(lr = self.critic_lr)
        
        self.last_state = None
        self.last_action = None
        
        # Initialize buffers
        self.buffer_old_logprob_v = []
        self.adv_buffer = []
        self.ref_buffer = []
        self.optimize_buffer = []
        self.distribution_buffer = []
        self.extra_index = [] # Focused training
        
    def agent_start(self,state,test=False):
        self.sum_rewards = 0
        self.episode_steps = 0
        self.total_turns = 0
        self.total_time = 0
        self.total_cost = 0
        self.last_state = state
        self.episode_buffer = []
        mu,log_std = self.actor(self.last_state)
        log_std = tf.clip_by_value(log_std, -9, 2)

        #####
        #print('AGENT_START')
        #print('state shape',state.shape)
        #print('state',state)
        #print('last state', self.last_state.shape)
        #print('mu shape',mu.shape)
        #print('mu',mu)
        #print('log_std shape',log_std.shape)
        #print('log_std',log_std)
        #####

        self.last_action = get_actions(mu, log_std)

        #####
        #print('last action shape',self.last_action.shape)
        #print('last action', self.last_action)
        #print('_______________')
        #####

        if test:
            self.last_action = mu
        else:
            self.distribution_buffer.append([mu.numpy(),log_std.numpy()])

        return self.last_action
        
    def agent_step(self, reward, state,test=False):    
        self.sum_rewards += reward
        self.episode_steps += 1

        mu,log_std = self.actor(state)
        log_std = tf.clip_by_value(log_std, -9, 2)

        if test:
            actions = mu
        else:
            actions = get_actions(mu, log_std)       
            # Append new experience to replay buffer
            self.episode_buffer.append([self.last_state, self.last_action, reward, 0, state])
            self.distribution_buffer.append([mu.numpy(),log_std.numpy()])

        #####
        #print('AGENT_STEP')
        #print('state shape', state.shape)
        #print('state',state)
        #print('reward',reward)
        #print('mu step shape', mu.shape, 'mu',mu)
        #print('log_std_step shape', log_std.shape,'log_std',log_std)
        #print('actions shape', actions.shape)
        #print('actions',actions)
        #print('episode_buffer',self.episode_buffer)
        #print('distribution_buffer',self.distribution_buffer)
        #print('_______________')
        #####

        # Update the last state and last action.
        self.last_state = state
        self.last_action = actions
    
    def agent_end(self, reward, state, test=False):
        self.sum_rewards += reward
        self.episode_steps += 1
        
        if not test:
            # Set terminal state to an array of zeros
            state = np.zeros_like(self.last_state)

            # Append new experience to replay buffer
            self.episode_buffer.append([self.last_state, self.last_action, reward, 1, state])

            # We get the advantages and references of all this episode
            adv_v, ref_v = get_adv_ref(self, self.episode_buffer)

            len_opt_before = len(self.optimize_buffer) # Focused training   
            
            self.adv_buffer.extend(adv_v) 
            self.ref_buffer.extend(ref_v) 
            self.optimize_buffer.extend(self.episode_buffer) 
            
            #####
            #print('AGENT END')
            #print('adv_v shape',np.shape(adv_v))
            #print('adv_v:',adv_v)
            #print('ref_v shape',np.shape(ref_v))
            #print('ref_v:',ref_v)
            #print('self.adv_buffer shape',np.shape(self.adv_buffer))
            #print('self.adv_buffer',self.adv_buffer)
            #print('self.ref_buffer shape',np.shape(self.ref_buffer))
            #print('self.ref_buffer',self.ref_buffer)
            #print('_______________')
            #####
            
            # Focused training
            if self.sum_rewards > 0:
                len_opt_after = len(self.optimize_buffer) 
                index = np.arange(len_opt_before,len_opt_after)
                index = index.tolist()
                if self.sum_rewards>200:
                    index = 4*index
                elif self.sum_rewards > 100:
                    index = 2*index
                self.extra_index.extend(index)

    def agent_message(self, message):
        if message == "get_sum_reward":
            return self.sum_rewards
        else:
            raise Exception("Unrecognized Message!")

In [13]:
def run_experiment(agent,num_episodes, max_count, load = False):
    env = gym.make(ENV_NAME)
    
    agent_sum_reward = np.zeros((1,num_episodes))
    agent_cost = np.zeros((1,num_episodes))
    agent_time = np.zeros((1,num_episodes))
    agent_turn = np.zeros((1,num_episodes))
    agent.agent_init(agent)
    
    if load:
        agent.critic.load_weights("G:/My Drive/JAXA/2020-2021/Transformer/weights/PPO_Lander_critic_" + str(108) + ".h5")
        agent.actor.load_weights("G:/My Drive/JAXA/2020-2021/Transformer/weights/PPO_Lander_actor_" + str(108) + ".h5")
        
    for episode in tqdm(range(1,num_episodes + 1)):
        run_episode(agent,env,test=False)
        episode_reward = agent.agent_message(agent,'get_sum_reward')
        agent_sum_reward[0,episode - 1] = episode_reward
        #agent_cost[0,episode - 1] = agent.total_cost
        #agent_time[0,episode - 1] = agent.total_time
        #agent_turn[0,episode - 1] = agent.total_turns
            
        # Perform a test 
        if episode % agent.test_episode == 0:
            print('Running test')
            count = 0
            sum_test_reward = 0
            for t in range(1,agent.test_iterations):
                run_episode(agent,env,test=True)
                test_reward = agent.agent_message(agent,'get_sum_reward')
                sum_test_reward += test_reward
                if test_reward > 200:
                    count += 1
                    print('Test:', t, 'Test_Reward:', test_reward)
                #print('Turns:', agent.total_turns, 'Time:', agent.total_time, 'Cost:', agent.total_cost)
            print('Test finished with success ' + str(count) + ' out of ' + str(agent.test_iterations) + ' iterations.')
            print('Test average reward is ' + str(sum_test_reward/agent.test_iterations))
            if count>=max_count:
                max_count = count
                agent.critic.save_weights("G:/My Drive/JAXA/2020-2021/Transformer/weights/PPO_Lander_critic_" + str(count) + ".h5")
                agent.actor.save_weights("G:/My Drive/JAXA/2020-2021/Transformer/weights/PPO_Lander_actor_" + str(count) + ".h5")
        
        run_episode(agent,env,test=False)
        
        if len(agent.optimize_buffer) > agent.buffer_size:
            mu_v, log_std_v = map(list, zip(*agent.distribution_buffer))

            #####
            #print('TRAIN INSIDE EPISODE')
            #print('mu before concat',np.shape(mu_v))
            #print('mu',mu_v)
            #print('log_std_v before concat', np.shape(log_std_v))
            #print('log_std',log_std_v)
            #print('len(agent.optimize_buffer)',len(agent.optimize_buffer))
            #####

            mu_v = np.concatenate(mu_v)
            mu_v = tf.convert_to_tensor(mu_v, dtype=tf.float32)
            log_std_v = np.concatenate(log_std_v)
            log_std_v = tf.convert_to_tensor(log_std_v, dtype=tf.float32)

            #####
            #print('mu after concat',mu_v.shape)
            #print('mu',mu_v)
            #print('log_std_v after concat', log_std_v.shape)
            #print('log_std',log_std_v)
            #####

            # Get states and actions
            states, actions, rewards, terminals, next_states = map(list, zip(*agent.optimize_buffer))

            actions = np.concatenate(actions)
            states = np.concatenate(states)

            #####
            #print('actions after concat',actions.shape)
            #print('actions',actions)
            #print('states after concat', states.shape)
            #print('states',states)
            #####

            old_logprob_v = calc_logprob(mu_v, log_std_v, actions)
            adv_buffer = np.array(agent.adv_buffer, dtype = np.float32) # size (sth,)
            adv_buffer = tf.expand_dims(adv_buffer,axis = 1).numpy()
            ref_buffer = np.array(agent.ref_buffer, dtype = np.float32) # size (sth,)
            ref_buffer = tf.expand_dims(ref_buffer,axis = 1).numpy()

            #####
            #print('old_logprob_v',old_logprob_v.shape)
            #print('adv_buffer', adv_buffer.shape)
            #print('ref_buffer', ref_buffer.shape)
            #####

            mean_adv = tf.math.reduce_mean(adv_buffer,axis =0, keepdims = True)
            std_adv = tf.math.reduce_std(adv_buffer,axis =0, keepdims = True)
            norm_adv = (adv_buffer - mean_adv) / (tf.clip_by_value(std_adv,1e-8,1e8)) # Corrected (this is the true normalization)
            #norm_adv = tf.keras.utils.normalize(adv_buffer,axis = 0) # Corrected
            
            idxs = np.arange(len(states))
            extra_idx = np.array(agent.extra_index) # Focused training
            if not len(extra_idx) == None: # Focused training
                idxs = np.append(idxs,extra_idx) # Focused training
            np.random.shuffle(idxs) 
            idxs = idxs.astype(int) # Focused training
            
            #####
            #print('norm_adv shape', norm_adv.shape)
            #print('norm_adv', norm_adv)
            #print('max of adv_v',tf.math.reduce_max(norm_adv))
            #print('min of adv_v',tf.math.reduce_min (norm_adv))
            #print('shape idx',idxs.shape)
            #print(idxs)
            #####
            
            for epoch in range(agent.epochs):
                #num_batches = int(math.floor(len(states)/agent.batch_size))
                num_batches = int(math.floor(len(idxs)/agent.batch_size)) # Focused training

                #####
                #print('NUM BATCHES', num_batches)
                #####

                for batch in range(num_batches):
                    start_num = batch*agent.batch_size
                    end_num = start_num + agent.batch_size

                    #####
                    #print('len(states)',len(states))
                    #print('shape states',states.shape)
                    #print('slice idx',idxs[start_num:end_num])
                    #####

                    #states_batch = states[start_num:end_num]
                    states_batch = np.array([states[idx] for idx in idxs[start_num:end_num]])
                    actions_batch = np.array([actions[idx] for idx in idxs[start_num:end_num]])
                    adv_batch = np.array([norm_adv[idx] for idx in idxs[start_num:end_num]]) 
                    ref_batch = np.array([ref_buffer[idx] for idx in idxs[start_num:end_num]]) 
                    old_logprob_v_batch = np.array([old_logprob_v[idx] for idx in idxs[start_num:end_num]])

                    #####
                    #print('BATCHES')
                    #print(' states_batch', states_batch.shape)
                    #print('actions_batch', actions_batch.shape)
                    #print('adv_batch', adv_batch.shape)
                    #print('ref_batch', ref_batch.shape)
                    #print('old_logprob_v_batch', old_logprob_v_batch.shape)
                    #print('adv_batch',adv_batch)
                    #####

                    optimize_network(agent,states_batch,actions_batch,adv_batch,ref_batch,old_logprob_v_batch)
                    
            #####
            #if episode%100 == 0:
                #print('mu_v', mu_v)
                #print('log_std_v',log_std_v)
            #####        
                    
            # restart all lists
            del agent.buffer_old_logprob_v[:]
            del agent.adv_buffer[:]
            del agent.ref_buffer[:]
            del agent.optimize_buffer[:]
            del agent.distribution_buffer[:]
            del agent.extra_index[:]

        if episode%10 == 0 or episode == 1:
            print('Episodes:', episode, 'Episodic_Reward:', episode_reward)
            #print('Turns:', agent.total_turns, 'Time:', agent.total_time, 'Cost:', agent.total_cost)
            
        if episode%50 == 0 or episode == 1:
            env.render()

        if episode_reward > 200:
          print('Success in episode ' + str(episode) + ' with a reward of ' + str(episode_reward))
        elif episode_reward > 100:
          print('Looking good in ' + str(episode) + ' with a reward of ' + str(episode_reward))
        elif episode_reward > 0:
          print('Getting somewhere in ' + str(episode) + ' with a reward of ' + str(episode_reward))
        
    save_name = "{}".format(agent.name) 
    if not os.path.exists('results'):
        os.makedirs('results')
    np.save("results/sum_reward_{}".format(save_name), agent_sum_reward)
    shutil.make_archive('results', 'zip', 'results')

In [14]:
def get_objective_attitude(min_ang, max_ang):
    euler_1 = (min_ang + rd.random()*(max_ang - min_ang)) * np.pi/180
    euler_2 = (min_ang + rd.random()*(max_ang - min_ang)) * np.pi/180
    euler_3 = (min_ang + rd.random()*(max_ang - min_ang)) * np.pi/180
    '''
    if rd.random() < 0.33:
        euler_1 = 0.0
    elif rd.random() > 0.66:
        euler_1 = 0.0
        euler_2 = 0.0
    '''
    initial_att = np.array([euler_1,euler_2,euler_3])
    
    # np.random.shuffle(initial_att)      
    objective_attitude = initial_att
    return objective_attitude

In [15]:
def run_episode(agent,env,test=False):
    done = False
    s = env.reset()

    # First the objective attitude is given in the form of Euler y-x-z in rad
    #objective_attitude = get_objective_attitude(MIN_ANG, MAX_ANG)
    # The initial attitude of the sc is always zero
    #initial_attitude = np.array([0,0,0])    
    # Initial state (1,6)
    #last_state = np.array([np.append(objective_attitude,np.append(initial_attitude,diff))])
    last_state = np.zeros((1,agent.state_size[0]))
    # Action (1,12)
    last_action = agent.agent_start(agent,last_state,test)
    # Let's start the real episode
    while (done == False):
        # We get the next state from the environment, as well as the reward and if terminal
        #state, reward, done = environment(agent)
        state, reward, done, _ = env.step(tf.squeeze(agent.last_action).numpy())
        state = np.array([state])#tf.expand_dims(state, axis=0).numpy().tolist()
        if done:
            agent.agent_end(agent,reward, state,test)
        else:
            agent.agent_step(agent, reward, state,test)
        

In [16]:
STATE_SIZE = (8,)
OUTPUT_SIZE = (2,)
TEST_ITERATIONS = 100
BATCH_SIZE = 64
ADAM_LR_CRITIC = 5e-4
ADAM_LR_ACTOR = 5e-5
GAMMA = 0.99
GAE_LAMBDA = 0.95
TOLERANCE = None
BUFFER_SIZE = 2048
TIMEOUT = None
PPO_EPS = 0.2
TEST_EPISODE = 50
TRAIN_EPOCHS = 10

In [17]:
PPO_agent = Agent
run_experiment(PPO_agent,3000, 0, load = False)

  0%|          | 0/3000 [00:00<?, ?it/s]

Episodes: 1 Episodic_Reward: -49.230032500672024


  0%|          | 10/3000 [00:08<1:07:56,  1.36s/it]

Episodes: 10 Episodic_Reward: -393.73575772356884


  1%|          | 20/3000 [00:16<49:14,  1.01it/s]  

...

Success in episode 913 with a reward of 270.8123953750912


 30%|███       | 914/3000 [1:12:23<1:42:34,  2.95s/it]

Success in episode 914 with a reward of 274.19803084604035


 30%|███       | 914/3000 [1:12:26<2:45:20,  4.76s/it]


KeyboardInterrupt: 

In [18]:
PPO_agent.critic.save_weights("G:/My Drive/JAXA/2020-2021/Transformer/weights/PPO_Lander_critic_BEST" + ".h5")
PPO_agent.actor.save_weights("G:/My Drive/JAXA/2020-2021/Transformer/weights/PPO_Lander_actor_BEST" + ".h5")

In [ ]:
PPO_agent.distribution_buffer


In [ ]:
env = gym.make(ENV_NAME)

In [ ]:
a = env.action_space.sample()

In [ ]:
PPO_agent.last_action

In [ ]:
a.shape

In [ ]:
b = tf.squeeze(PPO_agent.last_action).numpy()

In [ ]:
b.shape